### Import

In [59]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

### CSV 파일 로딩하기

In [61]:
file_path = './titanic.csv'
df = pd.read_csv(file_path)

### 결측치 확인

In [63]:
print("=== 결측치 현황 ===")
print(df.isnull().sum())

=== 결측치 현황 ===
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


### 결측치 처리 (Age인 경우 평균나이로) 

In [65]:
# age 컬럼의 평균값 계산 (결측치를 제외한 값으로 계산)
mean_age = df['Age'].mean()

# age 컬럼의 결측치를 평균값으로 채우기
df['Age'] = df['Age'].fillna(mean_age)

df = df.dropna(subset=['Embarked'])  # Embarked 결측치 제거

### 레이블 분포 확인

In [67]:
print("\n=== 레이블 분포 ===")
print(df['Survived'].value_counts())


=== 레이블 분포 ===
Survived
0    549
1    340
Name: count, dtype: int64


### 불필요한 칼럼 제거

In [69]:
cols_to_drop = ['PassengerId', 'Name', 'Ticket', 'Cabin'] #Cabin 컬럼 제거
df = df.drop(columns=cols_to_drop)

### 레이블 인코딩

In [71]:
# LabelEncoder 초기화
label_encoder = LabelEncoder()

# 인코딩할 컬럼 리스트
columns_to_encode = ['Sex', 'Embarked']  # 인코딩할 컬럼 리스트

#인코딩 전 'Survived' 컬럼 값의 분포 확인
print("\n=== Survived 컬럼 인코딩 전 ===")
print(df['Survived'].value_counts())

for column in columns_to_encode:
    df[column] = label_encoder.fit_transform(df[column])

#인코딩 후 'Survived' 컬럼 값의 분포 확인
print("\n=== Survived 컬럼 인코딩 후 ===")
print(df['Survived'].value_counts())


=== Survived 컬럼 인코딩 전 ===
Survived
0    549
1    340
Name: count, dtype: int64

=== Survived 컬럼 인코딩 후 ===
Survived
0    549
1    340
Name: count, dtype: int64


#### 해당 데이터셋에서는 Survived가 이미 0,1로 인코딩 되어있으므로, 인코딩 전후 차이는 없음

### 5가지 분류 (RF, DT, LR, KNN, SVM) 하고, accuracy 및 confusion matrix 확인

In [73]:
# 데이터 분할
X = df.drop('Survived', axis=1)
y = df['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 분류기 초기화
classifiers = {
    'Random Forest': RandomForestClassifier(),
    'Decision Tree': DecisionTreeClassifier(),
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'KNN': KNeighborsClassifier(),
    'SVM': SVC()
}

# 모델 학습 및 평가
results = []
for name, clf in classifiers.items():
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    results.append((name, acc, cm))

# 결과 출력
print("\n=== 모델 성능 비교 ===")
for name, acc, cm in results:
    print(f"\n{name}:")
    print(f"Accuracy: {acc:.4f}")
    print("Confusion Matrix:")
    print(cm)


=== 모델 성능 비교 ===

Random Forest:
Accuracy: 0.7697
Confusion Matrix:
[[86 23]
 [18 51]]

Decision Tree:
Accuracy: 0.7472
Confusion Matrix:
[[79 30]
 [15 54]]

Logistic Regression:
Accuracy: 0.7809
Confusion Matrix:
[[85 24]
 [15 54]]

KNN:
Accuracy: 0.7079
Confusion Matrix:
[[90 19]
 [33 36]]

SVM:
Accuracy: 0.6854
Confusion Matrix:
[[100   9]
 [ 47  22]]
